## Multi-task learning: Multi-label

Regression (2 labels)

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import random
import numpy as np
from tqdm import tqdm
import pandas as pd
from scipy import ndimage
from scipy.ndimage import zoom
import gc
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torch.backends.cudnn as cudnn
from models.custom_net import *
from models.sfcn_grade import *
from models.resnet import *
from models.inception_resnet_v2 import *
from models.densenet import *
import types
from scipy.stats import pearsonr, spearmanr
import torchio as tio
from datetime import datetime
import wandb
from models.ranking_loss import *
from models.focalloss import *

In [3]:
RANDOM_SEED = 551

In [4]:
# control randomness
def set_seed(random_seed=551):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    np.random.seed(random_seed)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(random_seed)

In [5]:
def load_data(img_dir, label_dir, std=False, norm=True):
    df = pd.read_csv(label_dir, index_col=0)
    filenames = df.index
    images = []
    for i, index in enumerate(filenames):
        # normalized images
        file_name = img_dir + index + '.npy'
        img = np.load(file_name)
        x, y, z = img.shape
        if std:
            m = np.mean(img)
            s = np.std(img)
            img = (img - m) / s
        if norm:
            img = (img - img.min()) / (img.max() - img.min())
        img = img.reshape((1, x, y, z))
        images.append(img)
    return images, df

In [6]:
# augmentation
def get_augmentation_transform():
    random_rotate = tio.RandomAffine(scales=(1.0, 1.0),
                                     degrees=12,)
    random_flip = tio.RandomFlip(axes='LR',
                                 flip_probability=0.5)
    random_shift = tio.RandomAffine(scales=(1.0, 1.0),
                                    degrees=0,
                                    translation=(20,20,20))
    compose = tio.transforms.Compose([random_rotate, random_flip, random_shift])
    augment = tio.transforms.OneOf([random_rotate, random_flip, random_shift, compose])
    return augment

In [7]:
class SEBlock(nn.Module):
    
    def __init__(self, in_channels, r=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool3d(1)
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels//r),
            nn.ReLU(),
            nn.Linear(in_channels//r, in_channels),
            nn.Sigmoid())
        
    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1, 1)
        return x

In [8]:
img_dir = 'img_npy/'
label_dir = 'labels/data_grade_foruse.csv'
label_name = ['Cerebral WM Hypointensities* Total Percent Of Icv', 'Cortical Gray Matter Total Percent Of Icv', 'GRADE']

In [9]:
weight_decay = 1e-6 #0.01
lr = 0.0001
epochs = 100
batch_size = 16
optimizer = optim.Adam
dropout_rate = 0.5
test_size = 0.2
# scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer=optimizer,
#                                                lr_lambda=lambda epoch: 0.3 ** epoch)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [11]:
class NeckDataset(Dataset):
    def __init__(self, index, X=None, y=None, transform=None):
        self.X = X
        self.y1 = y[index[0]].values  # 'Cerebral WM Hypointensities* Total Percent Of Icv'
        self.y2 = y[index[1]].values  # 'Cortical Gray Matter Total Percent Of Icv'
#         self.y3 = y[index[2]].values  # 'GRADE'
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = torch.FloatTensor(self.X[idx])
        label1 = torch.FloatTensor(self.y1)[idx]
        label2 = torch.FloatTensor(self.y2)[idx]
#         label3 = torch.LongTensor(self.y3)[idx]
        if self.transform is not None:
            image = self.transform(image)
#         return [image, label1, label2, label3]
        return [image, label1, label2]

In [12]:
def train(model, train_loader, lr, weight_decay, optim_class=optim.AdamW, scheduler=None, sorter='sodeep/weights/best_model_gruc.pth.tar'):
#     criterion1 = nn.L1Loss()
    criterion1 = rank_difference_loss(sorter)
#     criterion2 = nn.L1Loss()
    criterion2 = rank_difference_loss(sorter)
#     criterion3 = nn.CrossEntropyLoss()
#     criterion3 = FocalLoss(gamma=5)
        
    optimizer = optim_class(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    total_loss = 0
    correct = 0
    
    model.train()
    for inputs, labels1, labels2 in tqdm(train_loader):              
        
        # move data to the GPU
        inputs = inputs.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
#         labels3 = labels3.to(device)
        
        # clear previous gradient computation
        optimizer.zero_grad()

        # forward prop
        output1, output2 = model(inputs)
#         print('y: {} / y hat: {}'.format(labels, output))
            
        # calculate loss with cost function
        loss1 = criterion1(output1, labels1.unsqueeze(1).float())
        loss2 = criterion2(output2, labels2.unsqueeze(1).float())
#         loss3 = criterion3(output3, labels3)
        loss = loss1 + loss2
        
        # back prop
        loss.backward()
        
        # update model weights
        optimizer.step()
        if scheduler:
            scheduler.step()
        
        # accumulate loss
        total_loss += loss.data.item()
        
#         # accumulate correlation
        
#         # accumulate correct count
#         _, preds = torch.max(output3, 1)
#         correct += torch.sum(preds == labels.data)
                
        gc.collect()
        torch.cuda.empty_cache()
        
    return total_loss

In [13]:
def valid(model, valid_loader, sorter='sodeep/weights/best_model_gruc.pth.tar'):
#     criterion1 = nn.L1Loss()
    criterion1 = rank_difference_loss(sorter)
#     criterion2 = nn.L1Loss()
    criterion2 = rank_difference_loss(sorter)
#     criterion3 = nn.CrossEntropyLoss()
#     criterion3 = FocalLoss(gamma=5)
    
    total_loss = 0
    correct = 0
    
    model.eval()
    with torch.no_grad():
        for inputs, labels1, labels2 in tqdm(valid_loader):
            
            # move data to the GPU
            inputs = inputs.to(device)
            labels1 = labels1.to(device)
            labels2 = labels2.to(device)
#             labels3 = labels3.to(device)

            # forward prop
            output1, output2 = model(inputs)

            # calculate loss with cost function
            loss1 = criterion1(output1, labels1.unsqueeze(1).float())
            loss2 = criterion2(output2, labels2.unsqueeze(1).float())
#             loss3 = criterion3(output3, labels3)
            loss = loss1 + loss2
            
            # accumulate loss
            total_loss += loss.data.item()
            
#             # accumulate correct count
#             _, preds = torch.max(output, 1)
#             correct += torch.sum(preds == labels.data)
                    
    return total_loss

---

In [14]:
# load data
X, y = load_data(img_dir, label_dir, std=False, norm=True)  # std=False, norm=True

# initialize seed
set_seed()

# train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [15]:
# train set
train_set = NeckDataset(label_name, X_train, y_train)
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=16, shuffle=True)

In [16]:
# validation set
valid_set = NeckDataset(label_name, X_test, y_test)
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=16, shuffle=False)

In [ ]:
# # plot class distribution histogram
# plt.figure(figsize=(10,4))

# y_train, y_test = [], []
# for _, data in train_set:
#     y_train.append(data)
# for _, data in valid_set:
#     y_test.append(data)

# plt.subplot(1,2,1)
# plt.title('Class distribution in Train Set')
# plt.hist(y_train)

# plt.subplot(1,2,2)
# plt.title('Class distribution in Validation Set')
# plt.hist(y_test)

# plt.show()
# plt.close()

In [17]:
# only for classification
from torch.autograd import Variable

for images, _, _, labels3 in valid_loader:
    i, l = Variable(images), Variable(labels3)
    print(i.size())
    i = i.cpu().numpy()
    l = l.cpu().numpy()
    if l[0]==0:
        print('Label = {} : absent image'.format(l[0]))
    elif l[0]==1:
        print('Label = {} : punctate foci image'.format(l[0]))
    elif l[0]==2:
        print('Label = {} : beginning confluence image'.format(l[0]))
    else:
        print('Label = {} : large confluent areas image'.format(l[0]))
    plt.imshow(np.max(i[0].squeeze(), axis=1))
    plt.show()

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
class ShallowNet(nn.Module):

    def __init__(self, depth=1, classes=4):
        super(ShallowNet, self).__init__()
        
        self.feature_extractor = nn.Sequential()
        
        conv1 = nn.Sequential(
            nn.Conv3d(in_channels=depth, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(kernel_size=2, stride=2),
            nn.ReLU())
        
#         conv2 = nn.Sequential(
#             nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
#             nn.BatchNorm3d(64),
#             nn.MaxPool3d(kernel_size=2, stride=2),
#             nn.ReLU())
        
#         conv3 = nn.Sequential(
#             nn.Conv3d(in_channels=64, out_channels=64, kernel_size=1, stride=1),
#             nn.BatchNorm3d(64),
#             nn.ReLU())
        
        conv2 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm3d(64),
            nn.ReLU())
        
        self.feature_extractor.add_module('conv1', conv1)
        self.feature_extractor.add_module('conv2', conv2)
#         self.feature_extractor.add_module('conv3', conv3)
        
        self.classifier = nn.Sequential(
            nn.AvgPool3d([15,6,9]),
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv3d(in_channels=64, out_channels=64, kernel_size=1, stride=1)
        )
        
#         self.fc1 = nn.Linear(768, classes)
        self.fc1 = nn.Linear(9600, classes)

    def forward(self, x):
        
        x = self.feature_extractor(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [19]:
class ShallowNet2(nn.Module):

    def __init__(self, depth=1, classes=4):
        super(ShallowNet2, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels=depth, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(kernel_size=2, stride=2),
            nn.ReLU())
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU())
        
        self.classifier = nn.Sequential(
            nn.AvgPool3d([15,6,9]),
            nn.Dropout(p=0.5, inplace=False),
            nn.Conv3d(in_channels=64, out_channels=64, kernel_size=1, stride=1)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(11200, 11200),
            nn.ReLU(),
            nn.Linear(11200, 1))  # Cerebral WM Hypointensities* Total Percent Of Icv
        self.fc2 = nn.Sequential(
            nn.Linear(11200, 11200),
            nn.ReLU(),
            nn.Linear(11200, 1))  # Cortical Gray Matter Total Percent Of Icv
#         self.fc3 = nn.Sequential(
#             nn.Linear(11200, 11200),
#             nn.ReLU(),
#             nn.Linear(11200, classes))  # GRADE

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
#         x = x.view(-1, x.size(0))

        wmh_head = self.fc1(x)
        gm_head = self.fc2(x)
#         wmh_grade_head = self.fc3(x)
#         return wmh_head, gm_head, wmh_grade_head
        return wmh_head, gm_head

In [20]:
set_seed()

# #### model: custom simple net ####
# model = ShallowNet(depth=1, classes=4)
# ##################################

#### model: custom simple net ####
model = ShallowNet2(depth=1, classes=4)
##################################

# #### model: custom simple net ####
# model = CustomNet(depth=1, classes=4)
# ##################################

# #### model: simple net, last layer no activation function ####
# model = SFCN(output_dim=4)
# if dropout_rate != 0.5:
#     model.classifier.dropout.p = dropout_rate
# ###########################

# #### model: simple SE net ####
# model = SFCN()
# if dropout_rate != 0.5:
#     model.classifier.dropout.p = dropout_rate
# model.seblock = SEBlock(64)
# model.forward = types.MethodType(forward, model)
# ##############################

# #### model: resnet26 ####
# model = resnet26(in_channels=1, num_classes=4)
# #########################

# #### model: inception-resnet-v2 ####
# model = inception_resnet_v2(in_channels=1, num_classes=1)
# ####################################

# #### model: densenet ####
# model = densenet(121, in_channels=1, num_classes=1)
# #########################

model

ShallowNet2(
  (conv1): Sequential(
    (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (conv2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (classifier): Sequential(
    (0): AvgPool3d(kernel_size=[15, 6, 9], stride=[15, 6, 9], padding=0)
    (1): Dropout(p=0.5, inplace=False)
    (2): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  )
  (fc1): Sequential(
    (0): Linear(in_features=11200, out_features=11200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=11200, out_features=1, bias=True)
  )
  (fc2): Sequential(
    (0): Linear(in_features=11200, out_features=11200, bias=True)
    (1)

In [21]:
model.to(device)
wandb.init(project='multitask-test',
           config={"model": "shallowNet2-2reg", "dropout": dropout_rate, 
                   "lr": lr, "weight_decay": weight_decay, "epochs": epochs, "batch_size": batch_size,
                   "test_size": test_size})
wandb.watch(model)

best_rec = 0   # best_rec: best accuracy

set_seed()
for epoch in range(epochs):
    print('# Epoch %d / %d'%(epoch + 1, epochs))
    augment = get_augmentation_transform()
#     train_set = NeckDataset(X_train, y_train, transform=augment)
#     train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=16)

    # log image to check if augmentation is same for every experiment
    tensor_img = train_set[0][0].squeeze().cpu().detach().numpy()
    img = wandb.Image(np.max(tensor_img, axis=1), caption="Coronal MIP")

    loss_t = train(model, train_loader, lr, weight_decay, optimizer)
    loss_v = valid(model, valid_loader)

    train_loss = loss_t / len(train_loader)
    valid_loss = loss_v / len(valid_loader)

    wandb.log({"train_loss": train_loss, "valid_loss": valid_loss,
               "sample_img": img})
wandb.finish()
# torch.save(model.state_dict(), f'pretrained/221220_4class_shallow20.2x16_aug_epoch{epochs}')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hei-jung. Use `wandb login --relogin` to force relogin


# Epoch 1 / 100


100%|███████████████████████████████████████████| 11/11 [00:02<00:00,  5.22it/s]


# Epoch 2 / 100


100%|███████████████████████████████████████████| 11/11 [00:02<00:00,  4.88it/s]


...

# Epoch 99 / 100


100%|███████████████████████████████████████████| 11/11 [00:02<00:00,  5.19it/s]


# Epoch 100 / 100


100%|███████████████████████████████████████████| 11/11 [00:02<00:00,  4.59it/s]


train_loss,█▇▆▆▆▅▆▅▅▄▄▅▄▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▃▂▃▂▂▂▁▁▁▁▁▁
valid_loss,█▃▂▃▄▅▂▃▃▂▂▁▃▂▄▄▂▃▄▄▃▃▄▃▁▁▄▅▃▂▅▆▅▇▇▄▅▆▄▃
train_loss,1.52762
valid_loss,6.21472


In [22]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def predict(model, data_loader):
    model.eval()
    
    y_pred_list1, y_list1 = np.array([]), np.array([])
    y_pred_list2, y_list2 = np.array([]), np.array([])
#     y_pred_list3, y_list3 = np.array([]), np.array([])
    
    
    with torch.no_grad():
        for inputs, labels1, labels2 in tqdm(data_loader):
            
            # move data to the GPU
            inputs = inputs.to(device)
            labels1 = labels1.to(device)
            labels2 = labels2.to(device)
            
            # forward prop
            output1, output2 = model(inputs)

            # store predicted values
            pred = output1.detach().cpu().numpy()
            y_pred_list1 = np.append(y_pred_list1, pred.reshape(pred.size), axis=0)

            pred = output2.detach().cpu().numpy()
            y_pred_list2 = np.append(y_pred_list2, pred.reshape(pred.size), axis=0)
            
            # store truth values
            truth = labels1.cpu().detach().numpy()
            y_list1 = np.append(y_list1, truth.reshape(truth.size), axis=0)
            
            truth = labels2.cpu().detach().numpy()
            y_list2 = np.append(y_list2, truth.reshape(truth.size), axis=0)
            
    return y_list1, y_list2, y_pred_list1, y_pred_list2

In [23]:
y_list1, y_list2, y_preds1, y_preds2 = predict(model, valid_loader)

100%|███████████████████████████████████████████| 11/11 [00:02<00:00,  5.37it/s]


In [24]:
# label1
print(label_name[0])
pearson, _ = pearsonr(y_list1, y_preds1)
spearman, _ = spearmanr(y_list1, y_preds1)
print("Pearson:", pearson)
print("Spearman:", spearman)

Cerebral WM Hypointensities* Total Percent Of Icv
Pearson: 0.46102281474771584
Spearman: 0.5191741982256288


In [25]:
# label2
print(label_name[1])
pearson, _ = pearsonr(y_list2, y_preds2)
spearman, _ = spearmanr(y_list2, y_preds2)
print("Pearson:", pearson)
print("Spearman:", spearman)

Cortical Gray Matter Total Percent Of Icv
Pearson: 0.49759515127011816
Spearman: 0.47077007131354964


In [ ]:
#### model: custom simple net ####
model = ShallowNet(depth=1, classes=4)
model.to(device)
model.load_state_dict(torch.load('pretrained/221129_4classtest0.2x16noaug_epoch30'))
##################################